In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import plotly.express as px
from scipy import stats
import statsmodels.api as sm
import re
import math

In [2]:
from pandas_profiling import ProfileReport

In [3]:
data = pd.read_csv("Data Sets/investments_VC.csv", encoding = 'latin1')

In [4]:
pd.set_option("display.min_rows", 1000)
pd.set_option("display.max_columns", 100)

In [5]:
vc_df = pd.DataFrame(data)
vc_df.head(15)

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at,seed,venture,equity_crowdfunding,undisclosed,convertible_note,debt_financing,angel,grant,private_equity,post_ipo_equity,post_ipo_debt,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H
0,/organization/waywire,#waywire,http://www.waywire.com,|Entertainment|Politics|Social Media|News|,News,"17,50,000",acquired,USA,NY,New York City,New York,1.0,2012-06-01,2012-06,2012-Q2,2012.0,2012-06-30,2012-06-30,1750000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,/organization/tv-communications,&TV Communications,http://enjoyandtv.com,|Games|,Games,"40,00,000",operating,USA,CA,Los Angeles,Los Angeles,2.0,NaN,NaN,NaN,NaN,2010-06-04,2010-09-23,0.0,4000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,/organization/rock-your-paper,'Rock' Your Paper,http://www.rockyourpaper.org,|Publishing|Education|,Publishing,"40,000",operating,EST,NaN,Tallinn,Tallinn,1.0,2012-10-26,2012-10,2012-Q4,2012.0,2012-08-09,2012-08-09,40000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,/organization/in-touch-network,(In)Touch Network,http://www.InTouchNetwork.com,|Electronics|Guides|Coffee|Restaurants|Music|i...,Electronics,"15,00,000",operating,GBR,NaN,London,London,1.0,2011-04-01,2011-04,2011-Q2,2011.0,2011-04-01,2011-04-01,1500000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,/organization/r-ranch-and-mine,-R- Ranch and Mine,NaN,|Tourism|Entertainment|Games|,Tourism,"60,000",operating,USA,TX,Dallas,Fort Worth,2.0,2014-01-01,2014-01,2014-Q1,2014.0,2014-08-17,2014-09-26,0.0,0.0,60000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,/organization/club-domains,.Club Domains,http://nic.club/,|Software|,Software,"70,00,000",NaN,USA,FL,Ft. Lauderdale,Oakland Park,1.0,2011-10-10,2011-10,2011-Q4,2011.0,2013-05-31,2013-05-31,0.0,7000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7000000.0,0.0,0.0,0.0,0.0,0.0,0.0
6,/organization/fox-networks,.Fox Networks,http://www.dotfox.com,|Advertising|,Advertising,"49,12,393",closed,ARG,NaN,Buenos Aires,Buenos Aires,1.0,NaN,NaN,NaN,NaN,2007-01-16,2007-01-16,0.0,0.0,0.0,4912393.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,/organization/0-6-com,0-6.com,http://www.0-6.com,|Curated Web|,Curated Web,"20,00,000",operating,NaN,NaN,NaN,NaN,1.0,2007-01-01,2007-01,2007-Q1,2007.0,2008-03-19,2008-03-19,0.0,2000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,|Software|,Software,-,operating,USA,IL,"Springfield, Illinois",Champaign,1.0,2010-01-01,2010-01,2010-Q1,2010.0,2014-07-24,2014-07-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,/organization/01games-technology,01Games Technology,http://www.01games.hk/,|Games|,Games,"41,250",operating,HKG,NaN,Hong Kong,Hong Kong,1.0,NaN,NaN,NaN,NaN,2014-07-01,2014-07-01,41250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
vc_df.shape

(54294, 39)

In [7]:
vc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54294 entries, 0 to 54293
Data columns (total 39 columns):
permalink               49438 non-null object
name                    49437 non-null object
homepage_url            45989 non-null object
category_list           45477 non-null object
 market                 45470 non-null object
 funding_total_usd      49438 non-null object
status                  48124 non-null object
country_code            44165 non-null object
state_code              30161 non-null object
region                  44165 non-null object
city                    43322 non-null object
funding_rounds          49438 non-null float64
founded_at              38554 non-null object
founded_month           38482 non-null object
founded_quarter         38482 non-null object
founded_year            38482 non-null float64
first_funding_at        49438 non-null object
last_funding_at         49438 non-null object
seed                    49438 non-null float64
venture       

In [8]:
vc_df.isna().sum(axis = 0)

permalink                4856
name                     4857
homepage_url             8305
category_list            8817
 market                  8824
 funding_total_usd       4856
status                   6170
country_code            10129
state_code              24133
region                  10129
city                    10972
funding_rounds           4856
founded_at              15740
founded_month           15812
founded_quarter         15812
founded_year            15812
first_funding_at         4856
last_funding_at          4856
seed                     4856
venture                  4856
equity_crowdfunding      4856
undisclosed              4856
convertible_note         4856
debt_financing           4856
angel                    4856
grant                    4856
private_equity           4856
post_ipo_equity          4856
post_ipo_debt            4856
secondary_market         4856
product_crowdfunding     4856
round_A                  4856
round_B                  4856
round_C   

### Data Cleaning

#### Tasks
1. Delete empty rows
2. Delete irrelevant columns and ones with large number of missing values
3. Transform funding_total_usd
4. Clean founding_month, founding_quarter and founding_year
5. Delete all the missing rows for which major details are not given

#### Deleting empty rows

In [9]:
vc_df.dropna(axis = 0,how = 'all', inplace = True)

In [10]:
np.round(vc_df.isna().sum()/len(vc_df.index)*100,2)

permalink                0.00
name                     0.00
homepage_url             6.98
category_list            8.01
 market                  8.03
 funding_total_usd       0.00
status                   2.66
country_code            10.67
state_code              38.99
region                  10.67
city                    12.37
funding_rounds           0.00
founded_at              22.02
founded_month           22.16
founded_quarter         22.16
founded_year            22.16
first_funding_at         0.00
last_funding_at          0.00
seed                     0.00
venture                  0.00
equity_crowdfunding      0.00
undisclosed              0.00
convertible_note         0.00
debt_financing           0.00
angel                    0.00
grant                    0.00
private_equity           0.00
post_ipo_equity          0.00
post_ipo_debt            0.00
secondary_market         0.00
product_crowdfunding     0.00
round_A                  0.00
round_B                  0.00
round_C   

In [11]:
for i in list(vc_df.columns):
    if i.strip() != i:
        print(i)

 market 
 funding_total_usd 


In [12]:
vc_df.rename(columns = {" funding_total_usd ": "funding_total_usd",
                       " market " : "market"}, inplace = True, errors  = "raise")

#### Deleting Irrelevant Columns

In [13]:
vc_df.drop(labels = ["permalink","homepage_url", "state_code"], axis = 1, inplace = True)

#### Replacing '-' with NaN in Funding_total_usd column

In [14]:
vc_df["funding_total_usd"] = vc_df["funding_total_usd"].str.strip()

In [15]:
vc_df["funding_total_usd"].replace(to_replace = "-", value = 0, inplace = True )

In [16]:
vc_df["funding_total_usd"].replace(to_replace = r"[\,]", value = "", regex = True, inplace = True )

In [17]:
vc_df["funding_total_usd"] = vc_df["funding_total_usd"].astype(float)

In [18]:
np.round(vc_df.isna().sum()/len(vc_df.index)*100,2)

name                     0.00
category_list            8.01
market                   8.03
funding_total_usd        0.00
status                   2.66
country_code            10.67
region                  10.67
city                    12.37
funding_rounds           0.00
founded_at              22.02
founded_month           22.16
founded_quarter         22.16
founded_year            22.16
first_funding_at         0.00
last_funding_at          0.00
seed                     0.00
venture                  0.00
equity_crowdfunding      0.00
undisclosed              0.00
convertible_note         0.00
debt_financing           0.00
angel                    0.00
grant                    0.00
private_equity           0.00
post_ipo_equity          0.00
post_ipo_debt            0.00
secondary_market         0.00
product_crowdfunding     0.00
round_A                  0.00
round_B                  0.00
round_C                  0.00
round_D                  0.00
round_E                  0.00
round_F   

In [19]:
vc_df.drop(vc_df[vc_df["funding_total_usd"] == 0].index, inplace = True) 

In [20]:
np.round(vc_df.isna().sum()/len(vc_df.index)*100,2)

name                     0.00
category_list            6.12
market                   6.13
funding_total_usd        0.00
status                   2.70
country_code             9.34
region                   9.34
city                    11.01
funding_rounds           0.00
founded_at              21.28
founded_month           21.44
founded_quarter         21.44
founded_year            21.44
first_funding_at         0.00
last_funding_at          0.00
seed                     0.00
venture                  0.00
equity_crowdfunding      0.00
undisclosed              0.00
convertible_note         0.00
debt_financing           0.00
angel                    0.00
grant                    0.00
private_equity           0.00
post_ipo_equity          0.00
post_ipo_debt            0.00
secondary_market         0.00
product_crowdfunding     0.00
round_A                  0.00
round_B                  0.00
round_C                  0.00
round_D                  0.00
round_E                  0.00
round_F   

In [21]:
# Dropping the rows will no values in Market column
na_market = vc_df[vc_df["market"].isna()].index
vc_df.drop(na_market, inplace = True)

In [22]:
# Deleting rows with more than 5 empty columns
empty_5 = vc_df[vc_df.isna().sum(axis = 1)>5].index
vc_df.drop(empty_5, inplace = True)

In [23]:
vc_df["founded_at"] = pd.to_datetime(vc_df["founded_at"], yearfirst=True, errors = "coerce")
    

In [24]:
vc_df["founded_month"] = pd.DatetimeIndex(vc_df["founded_at"]).month
vc_df["founded_year"] = pd.DatetimeIndex(vc_df["founded_at"]).year
vc_df["founded_quarter"] = pd.DatetimeIndex(vc_df["founded_at"]).quarter
vc_df[["founded_year","founded_month","founded_quarter"]]

,founded_year,founded_month,founded_quarter
0,2012.0,6.0,2.0
1,NaN,NaN,NaN
2,2012.0,10.0,4.0
3,2011.0,4.0,2.0
4,2014.0,1.0,1.0
5,2011.0,10.0,4.0
6,NaN,NaN,NaN
7,2007.0,1.0,1.0
9,NaN,NaN,NaN
10,2012.0,1.0,1.0


In [25]:
vc_df.dropna(subset = ["region", "status"], inplace = True)

vc_df.drop(columns = "city" , inplace = True)

In [26]:
vc_df.drop(columns = list(vc_df.loc[:,"round_A":].columns), inplace = True)

### Data Visualization and EDA

In [27]:
vc_df.head(100)

,name,category_list,market,funding_total_usd,status,country_code,region,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at,seed,venture,equity_crowdfunding,undisclosed,convertible_note,debt_financing,angel,grant,private_equity,post_ipo_equity,post_ipo_debt,secondary_market,product_crowdfunding
0,#waywire,|Entertainment|Politics|Social Media|News|,News,1750000.0,acquired,USA,New York City,1.0,2012-06-01,6.0,2.0,2012.0,2012-06-30,2012-06-30,1750000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,&TV Communications,|Games|,Games,4000000.0,operating,USA,Los Angeles,2.0,NaT,NaN,NaN,NaN,2010-06-04,2010-09-23,0.0,4000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,'Rock' Your Paper,|Publishing|Education|,Publishing,40000.0,operating,EST,Tallinn,1.0,2012-10-26,10.0,4.0,2012.0,2012-08-09,2012-08-09,40000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,(In)Touch Network,|Electronics|Guides|Coffee|Restaurants|Music|i...,Electronics,1500000.0,operating,GBR,London,1.0,2011-04-01,4.0,2.0,2011.0,2011-04-01,2011-04-01,1500000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-R- Ranch and Mine,|Tourism|Entertainment|Games|,Tourism,60000.0,operating,USA,Dallas,2.0,2014-01-01,1.0,1.0,2014.0,2014-08-17,2014-09-26,0.0,0.0,60000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,.Fox Networks,|Advertising|,Advertising,4912393.0,closed,ARG,Buenos Aires,1.0,NaT,NaN,NaN,NaN,2007-01-16,2007-01-16,0.0,0.0,0.0,4912393.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,01Games Technology,|Games|,Games,41250.0,operating,HKG,Hong Kong,1.0,NaT,NaN,NaN,NaN,2014-07-01,2014-07-01,41250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,"1,2,3 Listo",|E-Commerce|,E-Commerce,40000.0,operating,CHL,Santiago,1.0,2012-01-01,1.0,1.0,2012.0,2013-02-18,2013-02-18,40000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,1-800-DOCTORS,|Health and Wellness|,Health and Wellness,1750000.0,operating,USA,Newark,1.0,1984-01-01,1.0,1.0,1984.0,2011-03-02,2011-03-02,0.0,0.0,0.0,0.0,1750000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,10 Minutes With,|Education|,Education,4400000.0,operating,GBR,London,2.0,2013-01-01,1.0,1.0,2013.0,2013-01-01,2014-10-09,400000.0,4000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
vc_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34492 entries, 0 to 49437
Data columns (total 27 columns):
name                    34492 non-null object
category_list           34492 non-null object
market                  34492 non-null object
funding_total_usd       34492 non-null float64
status                  34492 non-null object
country_code            34492 non-null object
region                  34492 non-null object
funding_rounds          34492 non-null float64
founded_at              28343 non-null datetime64[ns]
founded_month           28343 non-null float64
founded_quarter         28343 non-null float64
founded_year            28343 non-null float64
first_funding_at        34492 non-null object
last_funding_at         34492 non-null object
seed                    34492 non-null float64
venture                 34492 non-null float64
equity_crowdfunding     34492 non-null float64
undisclosed             34492 non-null float64
convertible_note        34492 non-null float64

In [30]:
vc_df[["category_list","market"]]

,category_list,market
0,|Entertainment|Politics|Social Media|News|,News
1,|Games|,Games
2,|Publishing|Education|,Publishing
3,|Electronics|Guides|Coffee|Restaurants|Music|i...,Electronics
4,|Tourism|Entertainment|Games|,Tourism
6,|Advertising|,Advertising
9,|Games|,Games
10,|E-Commerce|,E-Commerce
13,|Health and Wellness|,Health and Wellness
15,|Education|,Education


In [73]:
market_df = pd.DataFrame(vc_df["market"].value_counts()).reset_index()
market_df.rename(columns = {"index" : "sector", "market" : "number_of_companies"}, inplace = True)
market_df = market_df[market_df["number_of_companies"] > 100]

In [74]:
px.bar( x = market_df["sector"], y = market_df["number_of_companies"])